In [1]:
from flash.image import SemanticSegmentation
from flash.image import SemanticSegmentationData


In [2]:
%matplotlib inline
import cv2
import numpy as np
import torch
import cv2
from matplotlib import pyplot as plt
from flash import Trainer


In [3]:

model = SemanticSegmentation.load_from_checkpoint(r"E:\Google Drive\Acads\Notes\final sem\ML\project\learning-to-parse-pdf\documentAnalysis\flash\semantic_segmentation_model.pt")


Using 'efficientnet-b0' provided by qubvel/segmentation_models.pytorch (https://github.com/qubvel/segmentation_models.pytorch).
Using 'unet' provided by qubvel/segmentation_models.pytorch (https://github.com/qubvel/segmentation_models.pytorch).


In [4]:
dm = SemanticSegmentationData.from_folders(
    predict_folder=r"E:\Google Drive\Acads\Notes\final sem\ML\project\learning-to-parse-pdf\documentAnalysis\data\images/",
    transform_kwargs=dict(image_size=(800, 640)),
    batch_size = 1,
    )
trainer = Trainer(max_epochs=1, gpus = 1)
predictions = trainer.predict(model, dm)



GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

C:\Users\pveen\AppData\Local\Programs\Python\Python36\lib\site-packages\pytorch_lightning\trainer\data_loading.py:133: UserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


Predicting:   5%|▍         | 1/21 [00:04<01:28,  4.43s/it]

C:\Users\pveen\AppData\Local\Programs\Python\Python36\lib\site-packages\torch\nn\functional.py:3613: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


Predicting: 100%|██████████| 21/21 [00:08<00:00,  2.58it/s]


In [24]:
print('Total images', len(predictions))
for i in range(len(predictions)):
    print(predictions[i][0]['metadata'])
    inp = (predictions[i][0]['input']).numpy().copy()
    inp = np.transpose(inp, (1, 2, 0))
    inp = cv2.resize(inp, (640, 800))
    pred = (predictions[i][0]['preds'])
    pred = torch.softmax(pred, dim = 0)
    pred = torch.abs(pred)
    pred = torch.argmax(pred, dim = 0).numpy().astype(np.uint8)
    pred = cv2.resize(pred, (640, 800))
    inp = (inp/np.max(inp) * 255).astype(np.uint8)
    inp[:, :, 0][pred == 1] = 128
    inp[:, :, 2][pred == 5] = 128
    # inp[pred == 1] += 60
    # inp[pred == 2] += 90
    # inp[pred == 3] += 120
    # inp[pred == 4] += 150
    # inp[pred == 5] += 180
    # inp[pred == 6] += 210
    # inp[pred == 7] += 240

    

    b = inp
    c =  (pred/np.max(pred) * 255).astype(np.uint8)
    c = np.stack((c, c, c), axis = 2)
    out = np.hstack(( b, c))
    cv2.imwrite(f'preds/out_{i}.jpg', b)


Total images 21
{'size': [792, 601], 'filepath': 'E:\\Google Drive\\Acads\\Notes\\final sem\\ML\\project\\learning-to-parse-pdf\\documentAnalysis\\data\\images/PMC3576793_00004.jpg'}
{'size': [792, 601], 'filepath': 'E:\\Google Drive\\Acads\\Notes\\final sem\\ML\\project\\learning-to-parse-pdf\\documentAnalysis\\data\\images/PMC3654277_00006.jpg'}
{'size': [794, 596], 'filepath': 'E:\\Google Drive\\Acads\\Notes\\final sem\\ML\\project\\learning-to-parse-pdf\\documentAnalysis\\data\\images/PMC3777717_00006.jpg'}
{'size': [792, 601], 'filepath': 'E:\\Google Drive\\Acads\\Notes\\final sem\\ML\\project\\learning-to-parse-pdf\\documentAnalysis\\data\\images/PMC3863500_00003.jpg'}
{'size': [792, 612], 'filepath': 'E:\\Google Drive\\Acads\\Notes\\final sem\\ML\\project\\learning-to-parse-pdf\\documentAnalysis\\data\\images/PMC3866684_00003.jpg'}
{'size': [792, 601], 'filepath': 'E:\\Google Drive\\Acads\\Notes\\final sem\\ML\\project\\learning-to-parse-pdf\\documentAnalysis\\data\\images/PMC39